# Laboratory 3 – Dynamic queues modeling

In [ ]:
import time

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

### Customers distribution by Cashoers using FanOutPattern

![img](https://res.cloudinary.com/practicaldev/image/fetch/s--TSBm7yiw--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/75veyea6p6vynzpc3zse.png)

In [ ]:
LOG_PATH = "/data/notebook_files/logs"
DYNAMIC = "dynamic"
STATIC = "static"

## Metrics

$\text{Average waiting time} = \dfrac{\text{total time custumers wait in queue}}{\text{total number of customers}}$

$P_{wait} = \dfrac{\text{number of customers who wait}}{\text{total number of customers}}$

$P_{Idle \; Server} = \dfrac{total idle time of server}{total runtime of simulation}$

$\text{Average service time} = \dfrac{\text{total service time}}{total number of customers}$

In [ ]:
def getMetrics(logpath, type, min_cashiers, max_cashiers):
    start_time = time.time()

    df_cli = pd.read_csv(f"{logpath}/clients_{type}_{min_cashiers}.csv")
    df_serv = pd.read_csv(f"{logpath}/servers_{type}_{min_cashiers}.csv")

    metrics = {}
    waiters = df_cli[df_cli["ArrivedCashier"] - df_cli["ArrivedQueue"] > 0]
    metrics["avg_wait_time"] = (df_cli["ArrivedCashier"] - df_cli["ArrivedQueue"]).mean()
    metrics["prob_wait"] = len(waiters) / len(df_cli)
    metrics["avg_service_time"] = (df_cli["Left"] - df_cli["ArrivedCashier"]).mean()
    metrics["avg_wait_time_waiters"] = (waiters["ArrivedCashier"] - waiters["ArrivedQueue"]).mean()
    metrics["avg_sys_time"] = (df_cli["Left"] - df_cli["ArrivedQueue"]).mean()

    arrivals = []
    idles = []
    works = []

    # Preporcessing cashiers logs
    for i in range(max_cashiers):
        cur_cli = df_cli[df_cli["Cashier"] == i]
        cur_serv = df_serv[df_serv["Server"] == i]

        if len(cur_cli) == 0:
            continue

        arrivals.append(cur_cli.diff()["ArrivedCashier"].mean())

        to_drop = [idx for idx in cur_serv.index[cur_serv["Action"] == "DELETED"]]
        to_drop.extend([idx+1 for idx in to_drop])
        cur_serv.drop(to_drop, inplace=True)
        # left with created-started-finished-started-finished-...-finished
        #            odd[0]-even[0]- odd[1] -even[1]- odd[2] -...- odd[n]
        odd_serv = cur_serv.iloc[::2]["Time"].to_numpy()
        even_serv = cur_serv.iloc[1::2]["Time"].to_numpy()
        assert len(odd_serv) == len(even_serv)+1, "Server logs even after deleting DELETE-CREATE pairs"
        idle = (even_serv-odd_serv[:-1]).sum()
        work = (odd_serv[1:]-even_serv).sum()
        idles.append(idle)
        works.append(work)

    metrics["avg_time_arrivals"] = np.array(arrivals).mean()
    metrics["prob_idle"] = np.array(idles).sum() / (np.array(idles).sum() + np.array(works).sum())
    end_time = time.time()
    return metrics

In [ ]:
general_metrics_csh1 = getMetrics(LOG_PATH, STATIC, 1, 1)
general_metrics_csh1

{'avg_wait_time': 1.748,
 'prob_wait': 0.412,
 'avg_service_time': 2.84,
 'avg_wait_time_waiters': 4.242718446601942,
 'avg_sys_time': 4.588,
 'avg_time_arrivals': 4.943887775551103,
 'prob_idle': 0.4260307194826192}

In [ ]:
general_metrics_csh4 = getMetrics(LOG_PATH, STATIC, 4, 4)
general_metrics_csh4

{'avg_wait_time': 0.194,
 'prob_wait': 0.088,
 'avg_service_time': 2.836,
 'avg_wait_time_waiters': 2.2045454545454546,
 'avg_sys_time': 3.03,
 'avg_time_arrivals': 19.8694111487951,
 'prob_idle': 0.8544446725518374}

In [ ]:
dynamic_metrics_csh1 = getMetrics(LOG_PATH, DYNAMIC, 1, 5)
dynamic_metrics_csh1

{'avg_wait_time': 1.3236514522821576,
 'prob_wait': 0.34854771784232363,
 'avg_service_time': 2.8464730290456433,
 'avg_wait_time_waiters': 3.7976190476190474,
 'avg_sys_time': 4.170124481327801,
 'avg_time_arrivals': 5.108108108108108,
 'prob_idle': 0.44363341443633414}

In [ ]:
dynamic_metrics_csh4 = getMetrics(LOG_PATH, DYNAMIC, 4, 5)
dynamic_metrics_csh4

{'avg_wait_time': 0.184,
 'prob_wait': 0.076,
 'avg_service_time': 2.752,
 'avg_wait_time_waiters': 2.4210526315789473,
 'avg_sys_time': 2.936,
 'avg_time_arrivals': 18.971427509822167,
 'prob_idle': 0.854530077175177}

In [ ]:
metrics_slices = [
    general_metrics_csh1,
    general_metrics_csh4,
    dynamic_metrics_csh1,
    dynamic_metrics_csh4,
]

In [ ]:
def plotMetrics(metrics_slices):
    xs = ["Gen 1", "Gen 4", "Dyn 1", "Dyn 4"]
    for k in metrics_slices[0].keys():
        ys = []
        for metric in metrics_slices:
            ys.append(metric[k])

        fig = px.bar(x=xs, y=ys, color=xs)
        fig.update_layout(
            title_text=f"{k}"
        )
        fig.show()

In [ ]:
plotMetrics(metrics_slices=metrics_slices)

In [ ]:
def plotDynamics(logpath, min_cashiers, stop):
    df_serv = pd.read_csv(f"{logpath}/servers_{DYNAMIC}_{min_cashiers}.csv")

    start = df_serv.iloc[0]["Time"]
    end = df_serv.iloc[-1]["Time"]

    df_serv = df_serv[df_serv["Action"].isin(["CREATED", "DELETED"])]
    df_serv["CountCreate"] = (df_serv["Action"] == "CREATED").cumsum()
    df_serv["DeleteCount"] = (df_serv["Action"] == "DELETED").cumsum()
    df_serv["Count"] = df_serv["CountCreate"] - df_serv["DeleteCount"]

    X = list(df_serv["Time"])
    X.append(end)
    X = [item-start for item in X]
    X = np.array(X)

    # Working Ca
    Y = list(df_serv["Count"])
    Y.append(Y[-1])
    Y = np.array(Y)

    print(X)
    print(Y)

    fig = px.line(x=X, y=np.arange(start=0, stop=stop))
    fig.update_layout(
        title_text=f"Q start with {min_cashiers} cashier all time",
        yaxis_title="Active Cashiers",
        xaxis_title="ms passed",
    )
    fig.show()

    fig = px.line(x=X, y=Y)
    fig.update_layout(
        title_text=f"Pepole in que with {min_cashiers} cashier at start",
        yaxis_title="Active Cashiers",
        xaxis_title="ms passed",
    )
    fig.show()

    return X, Y

In [ ]:
X, Y = plotDynamics(LOG_PATH, 1, stop=10)

[   0   13   13  172  172  173  173  328  328 2466]
[1 2 1 2 1 2 1 2 1 1]


In [ ]:
X, Y = plotDynamics(LOG_PATH, 4, stop=5)

[   0    0    0    0 2384]
[1 2 3 4 4]
